In [58]:
import os
import numpy as np
import pandas as pd
import zipfile
import pandas as pd
import geopandas as gpd
from dbfread import DBF
from urllib.request import urlretrieve
from urllib.error import HTTPError

In [1]:
#Go back one level from notebook to data 
output_relative_dir = '../data/'

if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
for target_dir in ('curated','outer'):
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

### Download census data

In [2]:
url = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"#year-month.parquet
output_dir = "../data/outer/census.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [3]:
files = zipfile.ZipFile('../data/outer/census.zip','r')

for file in files.namelist():
    files.extract(file, f"../data/outer/census")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/07 23:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [88]:
types = ["A", "B"]
for type in types:
    G04= spark.read.option("header",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G04{type}_AUST_SA2.csv') #read the parquet 
    G04 = G04.na.drop(subset=G04.columns)
    G04.write.parquet(f"../data/curated/census_data_2021_04{type}.parquet")


In [92]:
types = ["A", "B", "C"]
for type in types:
    G17= spark.read.option("header",True).csv(f'../data/outer/census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G17{type}_AUST_SA2.csv') #read the parquet 
    G17 = G17.na.drop(subset=G17.columns)

    G17.write.mode("overwrite").parquet(f"../data/curated/census_data_2021_17{type}.parquet")

TypeError: 'str' object is not callable

In [6]:
url = "http://github.com/matthewproctor/australianpostcodes/zipball/master"#year-month.parquet
output_dir = "../data/outer/total_pto2.zip"
# download
urlretrieve(url, output_dir) 
print("complete")


complete


In [7]:
files = zipfile.ZipFile('../data/outer/total_pto2.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/total_pto2")

### Download postcode and SA2 data

In [82]:
post_sa2= spark.read.option("header",True).csv('../data/outer/total_pto2/matthewproctor-australianpostcodes-6f8a994/australian_postcodes.csv') 
print(post_sa2.dtypes)
post_sa2 = post_sa2.select("id", "postcode", "Lat_precise", "Long_precise", "SA2_MAINCODE_2016")
print(post_sa2.count() - post_sa2.na.drop().count())
post_sa2 = post_sa2.na.drop()
#177 rows with nulls dropped

#get the average latitude and longitude of the postcode if it have duplicates
post_sa2 = post_sa2\
    .withColumn("Lat_double", post_sa2["Lat_precise"].cast("double"))\
    .withColumn("Long_double", post_sa2["Long_precise"].cast("double"))
post_sa2 = post_sa2\
    .groupBy("postcode", "SA2_MAINCODE_2016")\
    .agg(F.avg("Lat_double").alias("avg_lat"), F.avg("Long_double").alias("avg_long"))

post_sa2.write.parquet("../data/curated/postcode_SA2.parquet")

[('id', 'string'), ('postcode', 'string'), ('locality', 'string'), ('state', 'string'), ('long', 'string'), ('lat', 'string'), ('dc', 'string'), ('type', 'string'), ('status', 'string'), ('sa3', 'string'), ('sa3name', 'string'), ('sa4', 'string'), ('sa4name', 'string'), ('region', 'string'), ('Lat_precise', 'string'), ('Long_precise', 'string'), ('SA1_MAINCODE_2011', 'string'), ('SA1_MAINCODE_2016', 'string'), ('SA2_MAINCODE_2016', 'string'), ('SA2_NAME_2016', 'string'), ('SA3_CODE_2016', 'string'), ('SA3_NAME_2016', 'string'), ('SA4_CODE_2016', 'string'), ('SA4_NAME_2016', 'string'), ('RA_2011', 'string'), ('RA_2016', 'string'), ('MMM_2015', 'string'), ('MMM_2019', 'string'), ('ced', 'string'), ('altitude', 'string'), ('chargezone', 'string'), ('phn_code', 'string'), ('phn_name', 'string'), ('lgaregion', 'string'), ('electorate', 'string'), ('electoraterating', 'string')]
177


### Download SA2 2021 information

In [9]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip" 
#year-month.parquet
output_dir = "../data/outer/2021sa2_shapefile.zip"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [10]:
files = zipfile.ZipFile('../data/outer/2021sa2_shapefile.zip','r')
for file in files.namelist():
    files.extract(file, f"../data/outer/2021sa2_shapefile")

### Download shapefile for each SA2 region
### !!!WARNING!!! THIS CODE IS GOING TO TAKE 30 MINUTES

In [70]:
path = r'../data/outer/2021sa2_shapefile/SA2_2021_AUST_GDA2020.dbf' # 文件目录
table = DBF(path)
geojson_option = "?_profile=oai&_mediatype=application/geo+json"
sa2_2021_temp = pd.DataFrame(iter(table))

shapes_list = []
for row in range(len(sa2_2021_temp)):
    url = sa2_2021_temp.iat[row,-1]
    try:
        shape = str(gpd.read_file(url + geojson_option).iat[0,-1])
    except HTTPError:
        shape = ""
    shapes_list.append(shape)

sa2_2021_temp["geometry"] = shapes_list


In [79]:
sa2_2021_selected = sa2_2021_temp[["SA2_CODE21", "SA2_NAME21", "geometry"]]
sa2_2021_selected.to_parquet("../data/curated/SA2_2021_AUST_GDA2020.parquet")

In [62]:
sa2_2021_temp

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.5842384630081 -35.4442571811303,..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POLYGON ((149.2189874391411 -35.36738117911253...
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POLYGON ((149.2132649330965 -35.34324521846067...
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POLYGON ((149.2403376383506 -35.34780977080727...
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POLYGON ((149.1957232435019 -35.36126247708308...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,(POLYGON ((150.6956683280878 -35.1829542884842...
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,(POLYGON ((167.9632532375536 -29.0721173752821...
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,


### Download file with 2016 SA2 info and 2021 SA2 info

In [23]:
url = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/correspondences/CG_SA2_2016_SA2_2021.csv"
output_dir = "../data/outer/correspondences.csv"
# download
urlretrieve(url, output_dir) 
print("complete")

complete


In [81]:
correspondences= spark.read.option("header",True).csv('../data/outer/correspondences.csv') #read the parquet 
correspondences = correspondences.na.drop()
correspondences.write.parquet("../data/curated/correspondences.parquet")
